### **Imports**

In [1]:
import pandas as pd

from sqlalchemy import create_engine
import secrets_
import urllib

### **Connecting with SQL Server database**

In [2]:
params = urllib.parse.quote_plus("""DRIVER={}; 
                                    SERVER={}; 
                                    DATABASE={}; 
                                    Trusted_Connection=yes""".format(secrets_.driver, secrets_.server, secrets_.database))

conn = create_engine("mssql+pyodbc:///?odbc_connect={}".format(params))

### **Analysis**

**SQL Query calculating daily retention rate**

In [3]:
pd.read_sql_query("""

-- unique combination of user and date
WITH user_data as (
	SELECT DISTINCT user_id, CAST(time_stamp AS date) as date_visited
	FROM data_visits
	WHERE time_stamp < '2021-06-11'),

-- number of users who visited platform on each day
daily_users as (
	SELECT date_visited, count(*) as users
	FROM user_data
	GROUP BY date_visited),

-- table with users, dates of their visits on platform and the next date when they returned to platofrm
users_dates AS (
	SELECT user_id, date_visited, 
			lead(date_visited, 1) over (partition by user_id ORDER BY  user_id, date_visited) AS date_returned
	FROM user_data),

-- number of users who returned to platform within 3 days from their previous visit for each day
returned_users AS (
	SELECT date_visited, count(*) as returned
	FROM users_dates
	WHERE DATEDIFF(day, date_visited, date_returned) IN (1,2,3)
	GROUP BY date_visited)

SELECT returned_users.date_visited, round((cast(returned as float)/users)*100, 2) as retention_rate
FROM returned_users INNER JOIN daily_users ON returned_users.date_visited = daily_users.date_visited
WHERE returned_users.date_visited <= '2021-06-07'
ORDER BY returned_users.date_visited
""", conn)

,date_visited,retention_rate
0,2021-06-01,70.17
1,2021-06-02,70.82
2,2021-06-03,70.22
3,2021-06-04,72.49
4,2021-06-05,69.69
5,2021-06-06,67.83
6,2021-06-07,68.57
